In [1]:
#Python Implementation of the Hartree Fock Method
#Procedures listed in the code follow as described in Modern Quantum Chemistry: 
#Introduction to Advanced Electronic Structure Theory, By Attila Szabo and Neil S. Ostlund
import math
import sys
import numpy as np
from molecule import atom
from molecule import vector
from molecule import gaussian
from molecule import molecule
from notebookImporter import importNotebook

#import integrals notebook for the hartree method
integrals = importNotebook("Hartree_Integrals")
scf = importNotebook("Hartree_SCF")

#define SCF convergence critera, and max number of iteration cycles
SCF_CONVERGENCE = pow(10, -100)
MAX_ITERATIONS = 100

#Step 1
#Specify Molecules, Nuclear Coordinates, and Charge of the nucli Number of Electrons,

#generate an h2 atom with a distance of 1.4 AU to compare with Szabo pg. 160
R = 1.4
system = molecule()
system.addAtom(atom(vector(1,1,1), 1, 1))
system.addAtom(atom(vector(1,1,1+R), 1,1))

#add a basis set
system.addBasis("6-21G")

system.display()

Molecule
Basis Set: 6-21G
Total Number of Electrons: 2
Atoms: 
     Atomic Number: 1, Electrons: 1, Coordinate: X: 1 Y: 1 Z: 1
     Atomic Number: 1, Electrons: 1, Coordinate: X: 1 Y: 1 Z: 2.4


In [2]:
#Step 2
#Calculate Integrals
#Overlap, KE, Nuclear Attraaction, and Electron Repulsion
S = integrals.overlap(system)
print("Overlap Matrix: ")
print(np.matrix(S))
print()

T = integrals.kineticEnergy(system)
print("Electron Kinetic Energy Matrix: ")
print(np.matrix(T))
print()

V = integrals.nuclearAttraction(system)
for index, atom in enumerate(V):
    print("Nucli " + str(index) + "-Electron Attraction Matrix: ")
    print(np.matrix(atom))
print()

electronRepulsion = integrals.electronElectronRepulsion(system)
print("Electron Repulsion Tensor: ")
print(np.array(electronRepulsion))
print()

#Form the electronic hamiltonian
H = np.matrix(T)

#add in all of the nuclear attractions matricies to the hamiltonian
for atom in V:
    H += np.matrix(atom)
        
print("Electronic Hamiltonian :")
print(H)
print()

Overlap Matrix: 
[[1.         0.64589812 0.40352864 0.48019147]
 [0.64589812 1.         0.48019147 0.83566476]
 [0.40352864 0.48019147 1.         0.64589812]
 [0.48019147 0.83566476 0.64589812 1.        ]]

Electron Kinetic Energy Matrix: 
[[1.54940495 0.29315085 0.21101261 0.17473316]
 [0.29315085 0.27478737 0.17473316 0.2021468 ]
 [0.21101261 0.17473316 1.54940495 0.29315085]
 [0.17473316 0.2021468  0.29315085 0.27478737]]

Nucli 0-Electron Attraction Matrix: 
[[-1.72091788 -0.78266155 -0.48198418 -0.56870018]
 [-0.78266155 -0.68300331 -0.3746172  -0.53836955]
 [-0.48198418 -0.3746172  -0.7078517  -0.44084143]
 [-0.56870018 -0.53836955 -0.44084143 -0.54946308]]
Nucli 1-Electron Attraction Matrix: 
[[-0.7078517  -0.44084143 -0.48198418 -0.3746172 ]
 [-0.44084143 -0.54946308 -0.56870018 -0.53836955]
 [-0.48198418 -0.56870018 -1.72091788 -0.78266155]
 [-0.3746172  -0.53836955 -0.78266155 -0.68300331]]

Electron Repulsion Tensor: 
[[[[1.14014487 0.61140189 0.38206058 0.44784188]
   [0.61

In [3]:
#Prepare for the SCF procedure

#get size of the basis set 
size = len(S)

#compute the Transformation Matrix
X = scf.X(S, size)

#get guess Fock matrix, assume 2-electron term is equal to 0
F = H

In [4]:
# SCF Procedure 

#init list to store the energy from each iteration
#as well as a boolean to signify whether the loop has converged
E = []
converged = False

while( not converged ):
  
    #diagnolze the Fock matrix and convert it to MO basis 
    F = X.transpose() * F * X 
    
    #diagnolize the Fock Matrix to obtain the MOs and the their respective energies
    MOEnergy, MO = np.linalg.eigh(F)
    
    #Transform the MO basis MOs to an AO basis
    C = X * MO
    
    #compute the electron density, the two electron term, and then use G to compute the new Fock matrix
    P = scf.densityMatrix(C, system.N, size)
    G = scf.G(electronRepulsion, P, size)
    F = H + G
    
    #compute the new expectation energy 
    E.append(scf.expectationEnergy(H, F, P))
    
    #check if at least two SCF iterations have occured
    #if more than two have occured, then check if the difference betweeen this E, 
    #and the previous E is less then the covergence value, if yes, end the SCF loop
    #if energy has not converged, check whether the max number of iterations have occured so far
    sizeE = len(E)
    if(len(E) > 2):
        if(abs(E[sizeE-2] - E[sizeE-1]) < SCF_CONVERGENCE):
            converged = True
        elif(sizeE > MAX_ITERATIONS):
            print("SCF Failed to Converge")
            break
    
    #compute total energy of the system including nuclear-nuclear repulsion
    totalE = E[sizeE-1] + scf.nuclearRepulsion(system)
    
    #display information about current SCF iteration to the user
    print("SCF Iteration #" + str(sizeE) + ", Electronic Energy: " + str(E[sizeE-1]) + " Hartrees, Total Energy: " + str(totalE) + " Hartrees")

SCF Iteration #1, Electronic Energy: -1.787325702337411 Hartrees, Total Energy: -1.0730399880516965 Hartrees
SCF Iteration #2, Electronic Energy: -1.8361123248275082 Hartrees, Total Energy: -1.121826610541794 Hartrees
SCF Iteration #3, Electronic Energy: -1.837195887668721 Hartrees, Total Energy: -1.1229101733830067 Hartrees
SCF Iteration #4, Electronic Energy: -1.8372185977435858 Hartrees, Total Energy: -1.1229328834578713 Hartrees
SCF Iteration #5, Electronic Energy: -1.8372190698823805 Hartrees, Total Energy: -1.122933355596666 Hartrees
SCF Iteration #6, Electronic Energy: -1.837219079686689 Hartrees, Total Energy: -1.1229333654009745 Hartrees
SCF Iteration #7, Electronic Energy: -1.8372190798902481 Hartrees, Total Energy: -1.122933365604534 Hartrees
SCF Iteration #8, Electronic Energy: -1.8372190798944745 Hartrees, Total Energy: -1.1229333656087603 Hartrees
SCF Iteration #9, Electronic Energy: -1.837219079894562 Hartrees, Total Energy: -1.1229333656088478 Hartrees
SCF Iteration #10